In [ ]:
from needle import plotHits, plotSubsetScores, plotNeedle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
import eden.graph as eg
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
from yoda import graphs as ygraphs
import needle
import yoda.alignments.clans as cl

from  sklearn.neighbors import KNeighborsClassifier
from kiez import Kiez


In [ ]:

import matplotlib as mpl
mpl.rc("figure", dpi=100)

In [ ]:
%%time
a,l = load_rfam(full= True, add_cov= False)


In [ ]:
# this is just to collect data to report in the paper
# asd= cl.get_label_dictionary(cl.clans_rfam15)
# asd= cl.get_label_dictionary(cl.clans)
# len(np.unique(list(asd.values()))) # 146
# len(asd) # 463

figpath =  !echo $HOME/pigplot/
figpath = figpath[0]
figpath


In [ ]:
import yoda.graphs as gr
matrix = gr.alignment_to_vectors(a)

In [ ]:
a2,l2 = load_rfam(full= True, add_cov= False)
mat = ygraphs.vectorize_alignments(a)

In [ ]:
# PLOT importance weighted vectorization  vs normal vectorization , do we have at least 1 NN in the closest X instances?
ax = plotHits(matrix, mat, l)
ax.get_figure().savefig(figpath + '4k_nspdk.png', bbox_inches='tight')

In [ ]:
# are there different successrates for more populated families?
manyseq, fewseq = filter_by_seqcount(a)
rf15Labels, burned  = cl.getlabels_rfam15(a)

# ax = plotSubsetScores(matrix, l, manyseq, fewseq, rf15Labels)
# ax.get_figure().savefig(figpath + '4k_alisize.png', bbox_inches='tight')
# plt.title('Hit Rate with full Rfam backdrop')

In [ ]:
sum([1 for i,e in enumerate(rf15Labels) if i not in burned and e != 0])

In [ ]:
# quick hack to compare the test set to the rest
from sklearn import metrics
pdist  = metrics.pairwise_distances(matrix)
# pdist = sml.CSLS(matrix)
print('all',sml.average_precision_nozero(pdist,l))
print('test',sml.average_precision(pdist,rf15Labels, burned))  # we will fix this in the future.. nozero doesnt know about the burn. this is annoying 
print('many',sml.average_precision_labelsubset(pdist,l, manyseq[:-1]))
print('few',sml.average_precision_labelsubset(pdist,l, fewseq[:-1]))
# print('random',np.mean([sml.average_precision_labelsubset(pdist,l, needle.randomhalf(l)) for x in range(20)]))


In [ ]:
import collections

z = collections.Counter(l)
l1 = [ z[e] for e in manyseq[:-1]] 
l2 = [ z[e] for e in fewseq[:-1]] 

plt.hist(l1)
plt.show()
plt.hist(l2)



# are two enough to find the rest?

In [ ]:


ax = needle.plotNeedle(matrix, l)
ax.get_figure().savefig(figpath + '4k_third.png', bbox_inches='tight')



In [ ]:
# prints the closest hits  for each clan
# needle.clanExtend(matrix,l,a,max=30)

# END OF USABLE STUFF


In [ ]:
cm_dict_ab={}
cm_dict_ba={
}
with open('rfam-weakpairs','r') as f:
    for line in f:
        if line:
            sline = line.split(' ')
            cm_dict_ab[sline[0]+sline[1]] = line
            cm_dict_ba[sline[1]+sline[0]] = line

asdasdasdasdasd

In [ ]:
vec = eg.Vectorizer( min_r = 2,min_d=1)
import structout.intlistV2 as so 

def nice(values, colorids=[]):
    symbols = '▁▂▃▄▅▆▇█'
    values = np.array(values)
    values = so.binning(values, count = len(symbols), ylim=False)
    color = lambda x: '\x1b[1;3%d;48m%s\x1b[0m' % (1, x)
    if not colorids: 
        return ''.join([symbols[v] for v in values])
    else:
        return ''.join([symbols[v] if i not in colorids else color(symbols[v]) for i,v in enumerate(values)])


r = []
done = set()
needles = []
rowz = []
for i,ind in enumerate(neigh_ind):
    # we want r0 and r1 to be equal and >0.. 
    rowlabels = yy[ind]
    if rowlabels[0] == 0 or rowlabels[0]!=rowlabels[1]:
       continue 
    # we want 
    c,d = ind[0], ind[1]
    if (c,d) in done:
        continue
    else:
        done.add((d,c))
    
    needle = np.array([ (a + b) /2  for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0])])
    needles.append(needle)
    rowz.append((i,c,d))
    #input = np.log1p(matrix[[c,d],:])
    #needle = np.asarray(np.sum(input, axis = 0))[0]
    #input = matrix[[c,d],:]
    #needle = np.asarray(np.sum(input, axis = 0))[0]
    # FAB
    #index_helper = np.array([((a+b)/2) if a and b else 0 for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0]) ])
    # needle = np.array([(a+b)/2 for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0]) if a and b])
    #indices = np.argsort(-np.dot(matrix.todense()[:, index_helper > 0] , needle.T )).A1 [:50]
    
    # dist, indices = k_inst.kneighbors([needle])
    
k_inst = Kiez(algorithm='SklearnNN', hubness='csls',  n_candidates = 100,  
              algorithm_kwargs= {'metric' : 'cosine'})
k_inst.fit(np.array(needles),matrix.toarray() )
dist, indices2 = k_inst.kneighbors()

for indices,rid in zip( indices2, rowz):
    i,c,d = rid
    # CLASS IDS 
    r.append(str(yy[indices]))
    print(r[-1])
    # RFAM IDS 
    # IMPORTANCES
    for i in indices[:10]: # FIST 20 HITS 
        graph = graphss[i]
        seqid_scr = {}
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            seqid_scr[n]= (ve*needle)[0]
        score_per_nucleotide = [ seqid_scr.get(j,0) for j in range(a[i].alignment.shape[1])]


        conserved = [ nid for nid,di in graphss[i].nodes(data=True) if di['weight']> .5]
        #### print per entry... 
        # NAME,  score 
        # position score + highlight 
        # structure line 
        # 5 ali lines 
        # newline 
        r.append(f"{a[i].gf['DE']} {a[i].gf['ID']}") # ! score later 
        # print(r[-1])
        r.append(f"{nice(score_per_nucleotide, conserved)}") 
        # print(r[-1])
        r.append(f"{a[i].gc['SS_cons']}") 
        # print(r[-1])
        for row in a[i].alignment[:5]:
            r.append(f'{"".join(row)}') 
        # print(r[-1])

        labels = a[c].label, a[i].label
        k = '%s.cm%s.cm' % labels
        r.append(cm_dict_ab.get(k,'None'))
        r.append(cm_dict_ba.get(k,'None'))
        labels = a[d].label, a[i].label
        k = '%s.cm%s.cm' % labels
        r.append(cm_dict_ab.get(k,'None'))
        r.append(cm_dict_ba.get(k,'None'))
        r.append(f'') 
        
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle
        return graph
    #gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    def draw(g):
        if len(g) < 400: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    # [draw(g) for g in gra]




In [ ]:
r

In [ ]:
np.array(needles).shape

In [ ]:
matrix.toarray().shape

In [ ]:
r[2]

In [ ]:
r = [e if e is not None else "None" for e in r]

In [ ]:
with open('data.txt','w') as file:
    file.write('\n'.join(r))
!cat data.txt | aha > data.html

In [ ]:
import sklearn
sklearn.__file__

In [ ]:
# CAN WE FIND NEW INSTANCES OF A CLUSTER IF WE PROVIDE THE WHOLE CLUSTER?

knn = KNeighborsClassifier(n_neighbors=100)
yy= np.array(l)
knn.fit(matrix ,yy)

y_needle =  {}

for y in np.unique(yy): 
    if y > 0:
        # pairwise search ... 
        indices = np.where (yy==y)[0]
        for a,b in [(i,j) for enu,i in enumerate(indices) for j in indices[enu+1:]]:
            input = np.log1p(matrix[[c,d],:])
            needle = np.asarray(np.sum(input, axis = 0))[0]           
        
    
        
        # input = matrix[yy == y]
        # needle = np.asarray(np.all((input > 0).todense(), axis = 0))
        
        dist, indices = knn.kneighbors(needle)
        print(y)
        print(yy[indices])
        print(dist)
        y_needle[y] = (needle, indices) 


In [ ]:
def show(y):
    needle, indices = y_needle[y]
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle[0]
        return graph
    gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    
    def draw(g):
        if len(g) < 300: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    [draw(g) for g in gra]


show(8)


In [ ]:
>>> rng = np.random.RandomState(0)
>>> source = rng.rand(61,65537)
>>> target = rng.rand(4108,65537)


# Step 2: Create a Kiez object
kiezz = Kiez(hubness='localscaling')  # 'ls' stands for Local Scaling, you can also use 'mp' for Mutual Proximity

# Step 3: Fit the Kiez object to your data
kiezz.fit(source)

# Step 4: Transform the data to get hubness-corrected distances
X_corrected = kiez.transform(source)

# Step 5: Convert the corrected data to a distance matrix
#distance_matrix = nn.kneighbors_graph(X_corrected, mode='distance').toarray()

In [ ]:
nn_ind.shap

In [ ]:
import numpy as np

a = np.arange(10)

a+=4

np.where(a < 6)[0]

In [ ]:
from scipy.stats import binom

In [ ]:
weights = [5,2,1]
values = [50, 10, 20]
z=[v for s, v in zip(weights, values) for _ in range(int(s))]

In [ ]:
np.mean(z), np.std(z)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score

# Generate test data
np.random.seed(0)  # For reproducibility
x = np.linspace(0, 10, 100)
y = np.concatenate((2 * x[:30], np.full(70,  10))) + np.random.normal(0, 1, x.shape)

# Calculate mutual information
# To compute mutual information, we need to discretize the continuous variables
# Here we use a simple binning approach
x_bins = np.histogram_bin_edges(x, bins=10)
y_bins = np.histogram_bin_edges(y, bins=10)

# Discretize x and y
x_digitized = np.digitize(x, x_bins)
y_digitized = np.digitize(y, y_bins)

# Calculate mutual information score
mi_score = mutual_info_score(x_digitized, y_digitized)

# Plot the data
plt.figure(figsize=(10, 6))
plt.scatter(x, y, color='blue', label='Data points', alpha=0.6)
plt.title('Test Data: y vs. x (Constant Second Half)')
plt.xlabel('x')
plt.ylabel('y')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
plt.grid()
plt.legend()
plt.show()

# Print the mutual information score
print(f'Mutual Information Score: {mi_score:.4f}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score

# Generate test data
np.random.seed(0)  # For reproducibility
x = np.linspace(0, 10, 100)

y = np.concatenate((2 * x[:50], 2 * (10 - x[50:]))) + np.random.normal(0, 1, x.shape)

# Calculate mutual information
# To compute mutual information, we need to discretize the continuous variables
# Here we use a simple binning approach
x_bins = np.histogram_bin_edges(x, bins=10)
y_bins = np.histogram_bin_edges(y, bins=10)

# Discretize x and y
x_digitized = np.digitize(x, x_bins)
y_digitized = np.digitize(y, y_bins)

# Calculate mutual information score
mi_score = mutual_info_score(x_digitized, y_digitized)

# Plot the data
plt.figure(figsize=(10, 6))
plt.scatter(x, y, color='blue', label='Data points', alpha=0.6)
plt.title('Test Data: y vs. x')
plt.xlabel('x')
plt.ylabel('y')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
plt.grid()
plt.legend()
plt.show()

# Print the mutual information score
print(f'Mutual Information Score: {mi_score:.4f}')